## Problema de la Mochila

(Knapsack problem)

### Solución cuántica a un problema clásico CQM con reformulación QUBO.

V1.8, Enero 2024

### El Problema clásico de la mochila

En el problema de mochila, se debe de empaquetar un conjunto de elementos de peso (o volumen) determinado en una mochila con una peso (o volumen) máximo. 

Además, estos elemenos tienen diferente valor para el mochilero, bien por su utilidad, bien por su coste

Si el tamaño o peso total de los elementos supera lo soportado por la mochila, no se pueden empaquetar todos. En ese caso, el problema es elegir un subconjunto de los elementos  que cabiendo en la mochila maximicen el valor de la carga.

Por otro lado, se puede limitar el número máximo de items de la misma clase que se pueden empaquetar para evitar soluciones de poca variabilidad (ausencia de items en la mochila).

A continuación se va a especificar un problema ejemplo, con 5 elementos como objetivo

<b> Parámetrización del problema</b>

    - Objetos:  N = 5 
    
    - Objetos máximos de una clase: n=2

    - Peso_max: w_max = 15
    
    - Pesos i : wi={1, 2, 1, 3, 2 }

    - Utilidad (1-5): ui={1, 3, 2, 5, 4}

Función objetivo: maximizar la utilidad asociada a los objetos en la mochila sujetos a la restricción del peso max

<b>Nota<b>
    
En los problemas CQM/BQM la función objetivo siempre se minimiza, de ahí los signos (-) en cqm.set_objective 

In [15]:
# Recursos
import dimod
from dimod import ConstrainedQuadraticModel, ExactCQMSolver, Integer, Binary
from dwave.system import DWaveSampler, EmbeddingComposite

# Funciones auxiliares

def sol_factibles(sampleset):
    # Resultados
    print("\nLas soluciones factibles al problema son:\n")
    print(sampleset.filter(lambda s: s.is_feasible).aggregate())
    
def sol_problema(result):
    # Agregación de los n reads
    samples = []
    ocurrencias = []

    for s in result.data():
        samples.append(invert(s.sample))
        ocurrencias.append(s.num_occurrences)

    sampleset = dimod.SampleSet.from_samples_cqm(samples,cqm,num_occurrences=ocurrencias)
    return(sampleset)


In [14]:
# PROBLEMA CQM
# Problema mochila  N items con máximo de n unidades por selecc.
# de utilidad ui y peso máximo w_max

N=5
n=2
w_max=15
wi=[1,2,1,3,2]
ui=[1,3,2,5,4]
 

# dimod.Integer con upper_bound <=n

rang_i= lambda s: int(w_max/s) if int(w_max/s) <= n else n 

x1 = dimod.Integer('x1', upper_bound=rang_i(wi[0]))
x2 = dimod.Integer('x2', upper_bound=rang_i(wi[1]))
x3 = dimod.Integer('x3', upper_bound=rang_i(wi[2]))
x4 = dimod.Integer('x4', upper_bound=rang_i(wi[3]))
x5 = dimod.Integer('x5', upper_bound=rang_i(wi[4]))


cqm = dimod.ConstrainedQuadraticModel()

cqm.set_objective(-ui[0]*x1-ui[1]*x2-ui[2]*x3-ui[3]*x4-ui[4]*x5)

cqm.add_constraint(wi[0]*x1+wi[1]*x2+wi[2]*x3+wi[3]*x4+wi[4]*x5 <= w_max, label='restricción peso max')


'restricción peso max'

In [11]:
print("Variables:")
print(cqm.variables)
print("Objetivo:")
print(cqm.objective)
print("Restricciones:") 
print(cqm.constraints)

Variables:
Variables(['x1', 'x2', 'x3', 'x4', 'x5'])
Objetivo:
ObjectiveView({'x1': -1.0, 'x2': -3.0, 'x3': -2.0, 'x4': -5.0, 'x5': -4.0}, {}, 0.0, {'x1': 'INTEGER', 'x2': 'INTEGER', 'x3': 'INTEGER', 'x4': 'INTEGER', 'x5': 'INTEGER'})
Restricciones:
{'restricción peso max': Le(ConstraintView({'x1': 1.0, 'x2': 2.0, 'x3': 1.0, 'x4': 3.0, 'x5': 2.0}, {}, 0.0, {'x1': 'INTEGER', 'x2': 'INTEGER', 'x3': 'INTEGER', 'x4': 'INTEGER', 'x5': 'INTEGER'}), 15.0)}


In [12]:
#Reformulación como problema qubo

#Uso de QP por defecto (DwaveSampler) y minorembedding (EmbeddingComposite)

sampler = EmbeddingComposite(DWaveSampler())

#Factor de Lagrange
fl=10

qubo, invert = dimod.cqm_to_bqm(cqm, lagrange_multiplier = fl)
result = sampler.sample(qubo, num_reads=1000)

In [13]:
sampleset=sol_problema(result)
sol_factibles(sampleset)


Las soluciones factibles al problema son:

    x1 x2 x3 x4 x5 energy num_oc. is_sat. is_fea.
0    2  1  1  2  2  -25.0       1 arra...    True
21   0  1  2  2  2  -25.0       2 arra...    True
1    1  1  1  2  2  -24.0       3 arra...    True
25   2  0  2  2  2  -24.0       1 arra...    True
118  1  2  2  1  2  -24.0       9 arra...    True
2    2  2  1  1  2  -23.0       1 arra...    True
27   2  1  2  2  1  -23.0       2 arra...    True
29   1  0  2  2  2  -23.0       3 arra...    True
33   0  1  1  2  2  -23.0       1 arra...    True
117  0  2  2  1  2  -23.0     150 arra...    True
3    0  2  1  2  1  -22.0       1 arra...    True
4    1  1  0  2  2  -22.0       2 arra...    True
5    1  2  1  1  2  -22.0      44 arra...    True
35   0  0  2  2  2  -22.0       1 arra...    True
37   2  2  2  2  0  -22.0       1 arra...    True
38   1  1  2  2  1  -22.0       1 arra...    True
6    0  2  1  1  2  -21.0     330 arra...    True
41   0  1  2  2  1  -21.0       2 arra...    True
43   1


<b>Comentarios</b>

- Con 1000 reads se encuentran 2 soluciones óptimas
- El parámetro n impacta mucho en la variabilidad de los items presentes en las soluciones.

<b>nota:</b>

- Con reads de aprox 100 ya se encuentran soluciones óptimas, pero se producen ausencias significativos.

- Una <b>métrica sencilla sobre la conveniencia de elevar los reads</b> es ver la moda de las soluciones óptimas. Si alguna es <=2, puede ser indicio de que alguna más pudiera haberse quedado fuera.


### Mejoras

Es trivial ampliar el problema para que contemple restricción de peso y volumen.

A continuación se repite el supuesto anterior con la restricción añadida de que los cinco objetos no superen el volumen de 10l.

También se va a <b>limitar el valor máximo de ajudicacion de los items a 2 unidades</b> (no se permite menos). Se podría modular esta asignación también mediante la variable de utilidad, pero por su sencillez, se opta por la limitación explícita.

- Botella de 1l agua: wi,vi,ui=(1,1,5)

- Bolsa 2Kg manzanas: wi,vi,ui=(3,4,4)

- Muda de ropa      : wi,vi,ui=(1,2,3)

- Equipo de foto    : wi,vi,ui=(2,2,2)

- Dron              : wi,vi,ui=(1,2,1)


In [22]:
# Problema mochila como problema CQM de variable discreta

N=5
n=2
w_max=6
v_max=9

wi=[1,3,1,2,1]
vi=[1,4,2,2,2]
ui=[5,4,3,2,1]
 
#Uso de QP

sampler = EmbeddingComposite(DWaveSampler())

# dimod.Integer impone que upper_bound >=2


x1 = dimod.Integer('x1', upper_bound=n)
x2 = dimod.Integer('x2', upper_bound=n)
x3 = dimod.Integer('x3', upper_bound=n)
x4 = dimod.Integer('x4', upper_bound=n)
x5 = dimod.Integer('x5', upper_bound=n)


cqm = dimod.ConstrainedQuadraticModel()

cqm.set_objective(-ui[0]*x1-ui[1]*x2-ui[2]*x3-ui[3]*x4-ui[4]*x5)

cqm.add_constraint(wi[0]*x1+wi[1]*x2+wi[2]*x3+wi[3]*x4+wi[4]*x5 <= w_max, label='restricción peso max')
cqm.add_constraint(vi[0]*x1+vi[1]*x2+vi[2]*x3+vi[3]*x4+vi[4]*x5 <= v_max, label='restricción vol max')



'restricción vol max'

In [23]:
print("Variables:")
print(cqm.variables)
print("Objetivo:")
print(cqm.objective)
print("Restricciones:") 
print(cqm.constraints)

Variables:
Variables(['x1', 'x2', 'x3', 'x4', 'x5'])
Objetivo:
ObjectiveView({'x1': -5.0, 'x2': -4.0, 'x3': -3.0, 'x4': -2.0, 'x5': -1.0}, {}, 0.0, {'x1': 'INTEGER', 'x2': 'INTEGER', 'x3': 'INTEGER', 'x4': 'INTEGER', 'x5': 'INTEGER'})
Restricciones:
{'restricción peso max': Le(ConstraintView({'x1': 1.0, 'x2': 3.0, 'x3': 1.0, 'x4': 2.0, 'x5': 1.0}, {}, 0.0, {'x1': 'INTEGER', 'x2': 'INTEGER', 'x3': 'INTEGER', 'x4': 'INTEGER', 'x5': 'INTEGER'}), 6.0), 'restricción vol max': Le(ConstraintView({'x1': 1.0, 'x2': 4.0, 'x3': 2.0, 'x4': 2.0, 'x5': 2.0}, {}, 0.0, {'x1': 'INTEGER', 'x2': 'INTEGER', 'x3': 'INTEGER', 'x4': 'INTEGER', 'x5': 'INTEGER'}), 9.0)}


In [24]:
#Reformulación como problema qubo

fl=10

qubo, invert = dimod.cqm_to_bqm(cqm, lagrange_multiplier = fl)
result = sampler.sample(qubo, num_reads=2000)

In [25]:
sampleset=sol_problema(result)
sol_factibles(sampleset)


Las soluciones factibles al problema son:

   x1 x2 x3 x4 x5 energy num_oc. is_sat. is_fea.
0   2  0  2  1  0  -18.0      14 arra...    True
1   2  1  1  0  0  -17.0       4 arra...    True
2   2  0  2  0  1  -17.0       9 arra...    True
3   2  0  2  0  0  -16.0      10 arra...    True
39  2  0  1  1  1  -16.0      16 arra...    True
4   2  0  1  1  0  -15.0      22 arra...    True
45  1  1  2  0  0  -15.0       6 arra...    True
70  2  1  0  0  1  -15.0       2 arra...    True
75  2  0  1  0  2  -15.0       1 arra...    True
5   1  0  2  1  1  -14.0      23 arra...    True
6   2  0  1  0  1  -14.0      13 arra...    True
7   2  0  0  1  2  -14.0       3 arra...    True
47  2  1  0  0  0  -14.0       6 arra...    True
50  2  0  0  2  0  -14.0       2 arra...    True
8   2  0  0  1  1  -13.0      16 arra...    True
9   1  1  1  0  1  -13.0       9 arra...    True
10  1  0  2  1  0  -13.0      32 arra...    True
52  1  0  2  0  2  -13.0       1 arra...    True
53  2  0  1  0  0  -13.0 


<b>Comentarios</b>

- Con 2000 reads se encuentran 14 soluciones óptimas
- fl > 15 empeora el rango de sol. óptimas


<b>nota:</b>

Con reads de aprox 100 ya se encuentran soluciones cuasi óptimas
